# Computational Cognitive Neuroscience Practical Assignment 4
## fMRI-based stimulus prediction using generative models
### Tommy Clausner (s4836219) and Steven Smits (s4237263)

### Importing libraries

In [ ]:
import scipy.io
import matplotlib.pyplot as plt
import numpy as np

### Importing data and preprocessing
[Make sure to have the data file ('69dataset.mat') located in the same folder as the script.]

In [2]:
### Data import
dataset69 = scipy.io.loadmat('69dataset.mat')
X = dataset69['X'] # Presented images. 100 images * 784 (28*28) pixels
Y = dataset69['Y'] # Measured brain activity. 100 corresponeding to images * 3092 (voxels)
X_prior = dataset69['prior'] # Image prior 2000 unshown stimuli * 784 pixels

# Data preparation
X_norm = (X - np.mean(X, axis=0)) / np.std(X, axis=0) # Normalize every pixel
X_norm[np.isnan(X_norm)] = 0 # Because some pixels are always 0, we get nan's for the denominator

Y_norm = (Y - np.mean(Y, axis=0)) / np.std(Y, axis=0) # Normalize every voxel
Y_norm[np.isnan(Y_norm)] = 0

X_prior_norm = (X_prior - np.mean(X_prior, axis=0)) / np.std(X_prior, axis=0) # Normalize every pixel
X_prior_norm[np.isnan(X_prior_norm)] = 0

X_train = np.concatenate((X_norm[0:40], X_norm[50:90])) # First 40 sixes, last 40 nines
X_test = np.concatenate((X_norm[40:50], X_norm[90:100])) # First 10 sixes, last 10 nines

Y_train = np.concatenate((Y_norm[0:40], Y_norm[50:90]))
Y_test = np.concatenate((Y_norm[40:50], Y_norm[90:100])) #6s [40:50], 9s [90:100]

/Users/Tommy/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:8: RuntimeWarning: invalid value encountered in divide
/Users/Tommy/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:14: RuntimeWarning: invalid value encountered in divide


### Defining default plotting function

In [3]:
# Own function for plot
def KAplot(x_estimate, t='', cols=4, rows=10):
    X_test_plot = X_test * np.std(X, axis=0) + np.mean(X,axis=0)
    fig = plt.figure()
    fig.suptitle(t, fontsize=14, fontweight='bold')
    rows = 10
    cols = 4
    index = 3
    a = fig.add_subplot(rows, cols, 1)
    plt.imshow(np.reshape(X_test_plot[0], [28, 28], order='F'), cmap='binary')  # Image from test data
    plt.axis('off')
    a.set_title('Original')
    a = fig.add_subplot(rows, cols, 2)
    plt.imshow(np.reshape(x_estimate[0], [28, 28], order='F'), cmap='binary')  # Order is for getting 6s/9s not mirrors
    plt.axis('off')
    a.set_title('Reconstructed')

    for i in range(1,20):  # Reconstruction of images based on B

            fig.add_subplot(rows, cols, index)
            plt.imshow(np.reshape(X_test_plot[i], [28, 28], order='F'), cmap='binary')
            plt.axis('off')
            index += 1
            fig.add_subplot(rows, cols, index)
            plt.imshow(np.reshape(x_estimate[i], [28, 28], order='F'), cmap='binary')
            plt.axis('off')
            index += 1

# Part 1

In [4]:
# Part 1
Lambda = 10**-6 # Lambda given
B = np.dot(np.dot(np.linalg.inv(np.identity(np.shape(np.dot(Y_train.T,Y_train))[0]) * Lambda + np.dot(Y_train.T,Y_train)), Y_train.T), X_train) # Ridge regression to get B in BX=Y
x_estimate = np.dot(B.T, Y_test.T).T

X_plot = (x_estimate * np.std(X, axis=0)) + np.mean(X,axis=0) # Undo normalization
KAplot(X_plot, t='Traditional approach')

# Part 2
The covariance is an unnormalized measure for the joint variability in the data. Hence, it represents the amount of information that is shared by certain pixels across the image.

In [5]:
# Part 2
B2 = np.dot(np.dot(np.linalg.inv(np.identity(np.shape(np.dot(X_train.T,X_train))[0]) * Lambda + np.dot(X_train.T,X_train)), X_train.T), Y_train) # Ridge regression to get B in BY = X
sigma_like = (10**-3) * np.identity(np.shape(Y_train)[1]) # Covariance matrix of likelihood
sigma_prior = np.dot(X_prior_norm.T, X_prior_norm) / (np.shape(X_prior_norm)[0] - 1) # Covariance matrix of prior
sigma_prior += (10**-6) * np.identity(np.shape(sigma_prior)[0]) # Covariance matrix  of prior, with tip to add 10^-6 to diagonal

plt.figure()
plt.imshow(sigma_prior) # Plot covariance matrix of prior
plt.axis('off');
plt.suptitle('Sigma matrix of prior', fontsize=14, fontweight='bold')

prec_prior = np.linalg.inv(sigma_prior) # Precision of prior
prec_like = np.linalg.inv(sigma_like) # Precision of likelihood


# Part 3
The Bayesian approach seems to work slightly worse than the traditional approach in terms of recognizability of the image reconstruction. This might be due to a bias in the prior. To elucidate, the prior contains two numbers towards the prediction can be biased. Thus, if a nine is predicted, the prior will slightly bias the results to a six and vice verse. This means that ambiguous fMRI data generates a pictures that is resembles the prior, which is a mix of nines and sixes. 

Improvements using a neural network approach:
One of the most prominent weaknesses of the presented approaches is their linearity. A neural network could overcome this problem by employing e.g. non-linear functions.

In [7]:
# Part 3
mu_post = np.dot(np.dot(np.dot(np.linalg.inv(np.dot(np.dot(B2,prec_like), B2.T) + prec_prior), B2), prec_like), Y_test.T).T

X_plot = mu_post * np.std(X, axis=0) + np.mean(X, axis=0) # Undo normalization
KAplot(X_plot, t='Bayesian approach')
